In [1]:
from arcgis.gis import GIS
gis = GIS("home")

## Part 1B - Create PRISM spacetime cube

### Set imports and file paths

In [3]:
import arcpy
import requests
import io
import os
import zipfile

In [76]:
#prep for download of prism data
prism_url = "https://www.prism.oregonstate.edu/fetchData.php?type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual"
prism_res = requests.get(prism_url)

#desktop paths
data_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2"
prism_path = os.path.join(data_path, "prism.zip")
extract_path = os.path.join(data_path, "prism")
gdb_path = os.path.join(data_path, "GIS5571_Lab2.gdb")

#laptop paths
#data_path = r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\GIS5571_Lab2"
prism_path = os.path.join(data_path, "prism.zip")
extract_path = os.path.join(data_path, "prism")
gdb_path = os.path.join(data_path, "GIS5571_Lab2.gdb")

In [77]:
#write to file
with open(prism_path, "wb") as file:
    file.write(prism_res.content)
    
#unzip to working folder
with zipfile.ZipFile(prism_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

### Convert bils to tiffs for mosaic compatibility

In [78]:
#create a list of the file names of just the bils so we can extract them
bil_files = [f for f in os.listdir(extract_path) if f.endswith('.bil')]

#the above code returns a list that includes the annual file I don't want. Let's take that out.

for file in bil_files:
    if "annual" in file.lower():
        bil_files.remove(file)

#check to make sure we have what we need
print(bil_files)

['PRISM_ppt_30yr_normal_4kmM4_01_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_02_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_03_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_04_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_05_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_06_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_07_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_08_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_09_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_10_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_11_bil.bil', 'PRISM_ppt_30yr_normal_4kmM4_12_bil.bil']


In [79]:
#create folder path for the tifs we'll use to convert to spacetime cube
tifs_folder = os.path.join(data_path, "prism_tifs")

#use bil_files list to convert just the files we want into tiffs
for bil in bil_files:
    bil_path = os.path.join(extract_path, bil) #this will create a new file name for each list entry
    #tif_name = bil.replace(".bil", ".tif")
    #tif_path = os.path.join(tifs_folder, tif_name)
    arcpy.conversion.RasterToOtherFormat(
    Input_Rasters= bil_path,
    Output_Workspace= tifs_folder,
    Raster_Format="TIFF"
)

### Create Mosaic Dataset and insert tiffs

In [85]:
#first we need to create the mosaic dataset - coordinate system is derived from the projection that bils (and now tifs) are in
arcpy.management.CreateMosaicDataset(
    in_workspace= gdb_path,
    in_mosaicdataset_name="PRISM_Normals_2",
    coordinate_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\gis5571_lab2.gdb\\PRISM_Normals_2'>

In [86]:
#using our folder of tiffs to fill out the mosaic dataset
#copying the python code from the tool mostly fills in the defaults - unnecessary code, so I've deleted some lines

arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="PRISM_Normals_2",
    raster_type="Raster Dataset",
    input_path=r"C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\prism_tifs",
    
)

<Result 'PRISM_Normals_2'>

### Prep multidimensional raster for conversion to spacetime cube

In [87]:
#add variable and timestamp fields to our footprint table - this is used in the next step

arcpy.management.CalculateField(
    in_table=r"PRISM_Normals_2\Footprint",
    field="Variable",
    expression='"precip"',
    expression_type="PYTHON3",
    field_type="TEXT",
)

arcpy.management.CalculateField(
    in_table=r"PRISM_Normals_2\Footprint",
    field="Timestamp",
    expression='DateAdd(Date(1990, 12, 1), $feature.OBJECTID-1, "month")',
    expression_type="ARCADE",
    field_type="DATE",
)

<Result 'PRISM_Normals_2\\Footprint'>

In [88]:
#converts variable and timestamp fields into correct format
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="PRISM_Normals_2",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'PRISM_Normals_2'>

In [89]:
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="PRISM_Normals_2",
    out_multidimensional_raster_layer="PRISM_Normals_MultidimLayer",
    variables="precip",
    template='-125.020833333 24.0625000000001 -66.4791666663333 49.9375000000001 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    )

<Result 'PRISM_Normals_MultidimLayer'>

In [91]:
cube_path = os.path.join(data_path, "PRISM_STC.nc")

arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="PRISM_Normals_MultidimLayer",
    output_cube= cube_path,
    fill_empty_bins="ZEROS"
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 110003: PRISM_STC.nc is not a valid Space Time Cube.
Failed to execute (VisualizeSpaceTimeCube3D).


In [94]:
arcpy.stpm.VisualizeSpaceTimeCube3D(
    in_cube= cube_path,
    cube_variable="PRECIP_NONE_ZEROS",
    display_theme="VALUE",
    output_features= os.path.join(gdb_path, "PRISM_STC_VisualizeSpaceTimeCube3D")
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\PRISM_STC_VisualizeSpaceTimeCube3D'>